# DiffForms

Train a neural network $f: \mathbb{R}^D \rightarrow \mathbb{R}$ that outputs values over a point cloud in such a way that the function varies smoothly over the manifold formed by the data

**Summary:**

1. Create an arbitrary point cloud:  

   $ X \in \mathbb{R}^{N \times D} $

2. Build a **k-NN graph** (k=6) with Euclidean distances

3. Compute the **heat kernel matrix**  

   $
   K_t(i,j) = \frac{1}{(4\pi t)^{d/2}} \exp\left( -\frac{\|x_i - x_j\|^2}{4t} \right)
   $

4. Compute a **discrete Laplacian** from the kernel (normalised or unnormalised)

5. Train a model to **approximate a function** that minimises a Laplacian regularisation:  

   $
   \mathcal{L}(f) = f^\top L f = \sum_{i,j} K_t(i,j) (f_i - f_j)^2
   $

_This encourages **smoothness** of \( f \) over the manifold._


In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Parameters
N = 100         # number of points
D = 3           # dimension of each point
k = 6           # k-NN
t = 1.0         # heat kernel time

# Step 1: Create random point cloud
X = torch.randn(N, D)

# Step 2: k-NN graph
nbrs = NearestNeighbors(n_neighbors=k + 1).fit(X)  # +1 includes the point itself
distances, indices = nbrs.kneighbors(X)

# Step 3: Build heat kernel matrix
heat_kernel = torch.zeros(N, N)

for i in range(N):
    """
    How heat would diffuse from poin i to point j over time t. 
    This favours local interactions, and is closely related to the Gaussian kernel. 
    The kernel is symmetric and positive, giving rise to a weighted adjacency matrix for the graph.
    """
    for j_idx, j in enumerate(indices[i][1:]):  # Skip self-loop
        d2 = torch.norm(X[i] - X[j]) ** 2
        coef = 1 / ((4 * np.pi * t) ** (D / 2))
        heat_kernel[i, j] = coef * torch.exp(-d2 / (4 * t))
        heat_kernel[j, i] = heat_kernel[i, j]  # symmetry


In [37]:
# Degree matrix
deg = torch.diag(heat_kernel.sum(dim=1))

# Unnormalised Laplacian
L = deg - heat_kernel

# Normalised Laplacian
# L = IDENTITY − deg^{-1/2} @ K @ D^{−1/2}

By optimising the Laplacian loss, the model learns a function that respects the geometry of the data. 

This is useful in many contexts and connects ideas from spectral graph theory, differential geometry, and machine learning.

In [38]:
class DiffForms(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, x):
        return self.net(x)

# Instantiate model
model = DiffForms(D, 32, 1)  # scalar-valued function on point cloud
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Laplacian smoothing loss
def laplacian_loss(f_vals, L):
    """
    This is the Dirichlet energy of f
    It encourages neighbouring points with strong kernel affinity to have similar function values. 
    Minimising this loss promotes smoothness of f over the manifold.
    """
    return torch.sum(f_vals.T @ L @ f_vals)

# Training loop
for epoch in range(500):
    optimizer.zero_grad()
    f_vals = model(X) # f : R^D -> R
    loss = laplacian_loss(f_vals, L)
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
        print(f"Epoch {epoch}: Laplacian loss = {loss.item():.6f}")


Epoch 0: Laplacian loss = 0.120950
Epoch 5: Laplacian loss = 0.020843
Epoch 10: Laplacian loss = 0.011319
Epoch 15: Laplacian loss = 0.007935
Epoch 20: Laplacian loss = 0.004639
Epoch 25: Laplacian loss = 0.002371
Epoch 30: Laplacian loss = 0.001802
Epoch 35: Laplacian loss = 0.001418
Epoch 40: Laplacian loss = 0.000736
Epoch 45: Laplacian loss = 0.000690
Epoch 50: Laplacian loss = 0.000494
Epoch 55: Laplacian loss = 0.000404
Epoch 60: Laplacian loss = 0.000301
Epoch 65: Laplacian loss = 0.000262
Epoch 70: Laplacian loss = 0.000223
Epoch 75: Laplacian loss = 0.000183
Epoch 80: Laplacian loss = 0.000157
Epoch 85: Laplacian loss = 0.000136
Epoch 90: Laplacian loss = 0.000120
Epoch 95: Laplacian loss = 0.000106
Epoch 100: Laplacian loss = 0.000095
Epoch 105: Laplacian loss = 0.000086
Epoch 110: Laplacian loss = 0.000079
Epoch 115: Laplacian loss = 0.000072
Epoch 120: Laplacian loss = 0.000066
Epoch 125: Laplacian loss = 0.000060
Epoch 130: Laplacian loss = 0.000054
Epoch 135: Laplacian lo